In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 21.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
import easyocr

In [ ]:
import re
from sklearn.metrics import f1_score

In [ ]:
input_csv = '/content/sample_test.csv'  # Replace with your input CSV file path
df = pd.read_csv(input_csv)

In [ ]:
# Step 4: Initialize EasyOCR Reader
reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [ ]:
results = []
for index, row in df.iterrows():
    img_url = row['image_link']  # Replace 'image_url' with your column name
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    text = reader.readtext(img, detail=0)
    results.append({'image_link': img_url, 'extracted_text': ' '.join(text)})

In [ ]:
# Step 6: Save Results to CSV
output_csv = 'outputEASYOCR.csv'  # Replace with your output CSV file path
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv, index=False)

print(f"Text extraction complete. Results saved to {output_csv}")

Text extraction complete. Results saved to outputEASYOCR.csv


In [ ]:
# Load the CSV data
data = pd.read_csv('/content/outputEASYOCR.csv')

# Define allowed units (add more as needed)
allowed_units = ['gram', 'centimetre', 'ounce', 'kilogram', 'meter', 'pound', 'litre', 'l', 'w', 'm', 'v', 'km', "inch", "mm", "M", 'MM', "V", 'KM']

# Regex to extract float numbers and units
def extract_number(text):
    match = re.search(r'\d+\.?\d*', text)  # Matches numbers like 12, 12.5
    if match:
        return float(match.group())
    return None

def extract_unit(text, allowed_units):
    for unit in allowed_units:
        if unit in text:
            return unit
    return None


In [ ]:
# Create new columns to extract number and unit
data['number'] = data['extracted_text'].apply(extract_number)
data['unit'] = data['extracted_text'].apply(lambda x: extract_unit(x, allowed_units))

# Function to combine number and unit into the required format "x unit"
def format_prediction(number, unit):
    if number is not None and unit is not None:
        return f"{number} {unit}"
    return None


In [ ]:

# Apply the function to create the predicted column
data['predicted_format'] = data.apply(lambda row: format_prediction(row['number'], row['unit']), axis=1)

# Save the results to a new CSV (optional)
data.to_csv('predicted_output2.csv', index=False)


